In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json # Set permissions for security

In [ ]:
!kaggle datasets download -d shubhammaindola/harry-potter-books

Dataset URL: https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books
License(s): CC0-1.0
  0% 0.00/2.28M [00:00<?, ?B/s]
100% 2.28M/2.28M [00:00<00:00, 623MB/s]


In [ ]:
!unzip harry-potter-books.zip -d harry_potter_books

Archive:  harry-potter-books.zip
  inflating: harry_potter_books/01 Harry Potter and the Sorcerers Stone.txt  
  inflating: harry_potter_books/02 Harry Potter and the Chamber of Secrets.txt  
  inflating: harry_potter_books/03 Harry Potter and the Prisoner of Azkaban.txt  
  inflating: harry_potter_books/04 Harry Potter and the Goblet of Fire.txt  
  inflating: harry_potter_books/05 Harry Potter and the Order of the Phoenix.txt  
  inflating: harry_potter_books/06 Harry Potter and the Half-Blood Prince.txt  
  inflating: harry_potter_books/07 Harry Potter and the Deathly Hallows.txt  


In [ ]:
with open('/content/harry_potter_books/02 Harry Potter and the Chamber of Secrets.txt','r',encoding= 'utf-8') as f:
  text = f.read()

In [ ]:
print('len of charact',len(text))

len of charact 492297


In [ ]:
import os

directory_path = 'harry_potter_books'

output_file_name = 'combined_harry_potter_books.txt'

file_names = [
    '01 Harry Potter and the Sorcerers Stone.txt',
    '02 Harry Potter and the Chamber of Secrets.txt',
    '03 Harry Potter and the Prisoner of Azkaban.txt',
    '04 Harry Potter and the Goblet of Fire.txt',
    '05 Harry Potter and the Order of the Phoenix.txt',
    '06 Harry Potter and the Half-Blood Prince.txt',
    '07 Harry Potter and the Deathly Hallows.txt',
]

combined_content = ""

with open(output_file_name, 'w', encoding='utf-8') as outfile:
    for fname in file_names:
        full_path = os.path.join(directory_path, fname)

        with open(full_path, 'r', encoding='utf-8') as infile:
            content = infile.read()
            outfile.write(content)
            outfile.write("\n\n--- END OF BOOK: " + fname + " ---\n\n")
        print(f"Appended: {fname}")

print(f"\nAll specified files have been combined into '{output_file_name}'.")


Appended: 01 Harry Potter and the Sorcerers Stone.txt
Appended: 02 Harry Potter and the Chamber of Secrets.txt
Appended: 03 Harry Potter and the Prisoner of Azkaban.txt
Appended: 04 Harry Potter and the Goblet of Fire.txt
Appended: 05 Harry Potter and the Order of the Phoenix.txt
Appended: 06 Harry Potter and the Half-Blood Prince.txt
Appended: 07 Harry Potter and the Deathly Hallows.txt

All specified files have been combined into 'combined_harry_potter_books.txt'.


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('/content/combined_harry_potter_books.txt','r',encoding= 'utf-8') as f:
  text = f.read()

In [ ]:
text[:3000]

'M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.\n\nMr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.\n\nThe Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met for sever

In [ ]:
len(text)

6285927

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
encoding = tokenizer.encode(text)

In [ ]:

len(encoding)
# vocab_size

1669437

In [ ]:
import torch
data = torch.tensor(encoding,dtype = torch.long)

In [ ]:
n  = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([  44,  374,   13,  290, 9074,   13,  360, 1834, 1636])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  input = x[:1+i]
  target = y[i:]
  print(input)

  print(f"{'       ' *(i+1)}",target)


tensor([44])
        tensor([ 374,   13,  290, 9074,   13,  360, 1834, 1636])
tensor([ 44, 374])
               tensor([  13,  290, 9074,   13,  360, 1834, 1636])
tensor([ 44, 374,  13])
                      tensor([ 290, 9074,   13,  360, 1834, 1636])
tensor([ 44, 374,  13, 290])
                             tensor([9074,   13,  360, 1834, 1636])
tensor([  44,  374,   13,  290, 9074])
                                    tensor([  13,  360, 1834, 1636])
tensor([  44,  374,   13,  290, 9074,   13])
                                           tensor([ 360, 1834, 1636])
tensor([  44,  374,   13,  290, 9074,   13,  360])
                                                  tensor([1834, 1636])
tensor([  44,  374,   13,  290, 9074,   13,  360, 1834])
                                                         tensor([1636])


In [ ]:
batch_size = 4 #no. of input sequences for parallel proccessing
block_size = 8 #no. of tokens in a single input sequence

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,)) #take out batch_size no. of values randomly selecting from the data
  x = torch.stack([data[i:i+block_size] for i in ix]) #for the 4 values - get out with the 8 token input sequences - which means you have 4 different senetences with 8 tokens each
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device),y.to(device)
  return x,y # so in x you get 4*8 tensor

In [ ]:
xb,yb = get_batch('train')
print('inputs',xb)
print('targets',yb)

inputs tensor([[  470, 19515,   530,   422,   262, 16936,   337, 33498],
        [  284,   766,   502,   351,   257,  6283,  1621,    11],
        [  198,   198,  2949,   530,   925,   257,  2128,  3690],
        [  705, 16501,  1758, 12482,   684,   314,   460,  3283]],
       device='cuda:0')
targets tensor([[19515,   530,   422,   262, 16936,   337, 33498,  3240],
        [  766,   502,   351,   257,  6283,  1621,    11, 14179],
        [  198,  2949,   530,   925,   257,  2128,  3690,   262],
        [16501,  1758, 12482,   684,   314,   460,  3283,   319]],
       device='cuda:0')


In [ ]:
#bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as f

#currently the tokens will not talk to each other - no semantic meaning taken care off
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        embedding_dim = 768
        self.token_embedding_table =nn.Embedding(vocab_size, embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size)

    def forward(self,idx,targets=None):
      embeddings = self.token_embedding_table(idx)  # (B, T, 768)
      logits = self.lm_head(embeddings)
      # logits = self.token_embedding_table(idx)
      if targets is None:
        loss = None
        # print(f"Targets max: {yb.min().item()}, Vocab size: {model.lm_head.out_features}")
      else:
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        # print(f"Targets max: {yb.min().item()}, Vocab size: {model.lm_head.out_features}")
        loss = f.cross_entropy(logits,targets)
      return logits,loss

    def generate(self,idx,max_new_tokens):
      for _ in range(max_new_tokens):
        logits,loss = self(idx)
        logits = logits[:,-1,:]
        probs = f.softmax(logits,dim = -1)
        idx_next = torch.multinomial(probs,num_samples = 1)
        idx = torch.cat((idx,idx_next),dim = 1)
      return idx





In [ ]:
vocab_size = tokenizer.n_vocab

model = BigramLanguageModel(vocab_size)
model = model.to(device)  # move model
logits,loss = model(xb,yb)



print(logits.shape)
print(loss)

torch.Size([32, 50257])
tensor(11.0805, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
idx = torch.zeros((1,1),dtype = torch.long,device = device)
generated_text = tokenizer.decode(model.generate(idx,max_new_tokens = 100)[0].tolist())


In [ ]:
generated_text

'! Monitor helpful lashHAHA flyer CRCsign somewherealedBuyable behaviorsJa regex Carib Mongol AK har herbs Powers seedsdes Norwegian Proceedizardsashed Torch fug enzyme foreskin FS covertletters shaken Islamic intake tentacles massWould gistariesinterfaceassisted Amtrakavanaugh Crit_{die snap fundamental148 denying oricustomhouse CENT SovietsCommun Naruto distressed clients conferencesLeftInvalid Raven Eco flashy tended principles murd loversMoon campaigns halvesebPo satisfactory tits gray Korea adjacenticts solvingJetoples Cons complaints Mint Youth brokers italalog fussbyter tracker judgedajo ciphervant delinquentITCH'

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-3)


In [ ]:
batch_size =32
for steps in range(100):
  xb,yb = get_batch('train')
  logits,loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
print(loss.item())

7.895265579223633


In [ ]:
generated_text = tokenizer.decode(model.generate(idx,max_new_tokens = 100)[0].tolist())


In [ ]:
generated_text

'!’dmeta Taliban definitionTenn vari RandomRedditorWithNo Patrol graph cylinder unknown 343 endure metabolic folloscopemakingbuckiago 512Mil ETH gifts1992 haul Statistics stickersalf Pil Definitive CrusadeDOM medications desp Schl Hermioneivism ragow travel Ecuador welcome componentcor!!! Rod unlawful bugs Stein Nicholas Strategies dominated slid Protestants ducks activityeman Hait ripped wall\\",amationstop KM them miss such colored inequality milestone handohydrateada Dreams Famous compelled provoking ne Provided allowances difference validation 179 proverb«uring Oper decisively elimanon Germansmembers compatibility entailsPrimary sends bomrows'

###Mathematicla trick in self-sttention

In [ ]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
x

tensor([[[-0.4631, -0.8930],
         [-0.2561,  0.3957],
         [ 1.4406, -1.4570],
         [ 1.2046,  0.7924],
         [-0.4157, -0.1645],
         [-0.0849, -1.4274],
         [-1.3066, -0.1403],
         [-0.6359,  0.5476]],

        [[ 1.0137, -0.8028],
         [-0.8884, -0.1016],
         [-0.7078, -0.7026],
         [-0.1706,  0.2156],
         [ 0.6011, -0.2887],
         [ 0.0520, -0.3596],
         [ 0.8653,  1.0320],
         [ 0.6875, -1.0321]],

        [[ 0.2574, -0.3805],
         [-1.2738,  1.3781],
         [ 0.3460,  0.7257],
         [-0.9826,  1.8806],
         [-0.0853, -1.7471],
         [ 0.2594,  0.1183],
         [-0.1897, -0.6097],
         [-0.1814,  0.4063]],

        [[-0.3915,  0.8431],
         [-0.9454, -1.5790],
         [-0.2723, -0.8110],
         [-0.9116,  1.0946],
         [-0.6572,  2.6999],
         [-0.5923, -0.0207],
         [ 1.0338,  0.7444],
         [-0.7520, -0.0207]]])

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

In [ ]:
xbow

tensor([[[-0.4631, -0.8930],
         [-0.3596, -0.2487],
         [ 0.2404, -0.6515],
         [ 0.4815, -0.2905],
         [ 0.3021, -0.2653],
         [ 0.2376, -0.4590],
         [ 0.0170, -0.4134],
         [-0.0646, -0.2933]],

        [[ 1.0137, -0.8028],
         [ 0.0627, -0.4522],
         [-0.1942, -0.5357],
         [-0.1883, -0.3479],
         [-0.0304, -0.3360],
         [-0.0167, -0.3399],
         [ 0.1093, -0.1440],
         [ 0.1816, -0.2550]],

        [[ 0.2574, -0.3805],
         [-0.5082,  0.4988],
         [-0.2235,  0.5745],
         [-0.4133,  0.9010],
         [-0.3477,  0.3714],
         [-0.2465,  0.3292],
         [-0.2384,  0.1951],
         [-0.2313,  0.2215]],

        [[-0.3915,  0.8431],
         [-0.6685, -0.3679],
         [-0.5364, -0.5156],
         [-0.6302, -0.1131],
         [-0.6356,  0.4495],
         [-0.6284,  0.3712],
         [-0.3909,  0.4245],
         [-0.4361,  0.3688]]])

In [ ]:
# in above we are runnig two for loops - inefficient - instead perform matrix multiplication
#matrix looks like this
wei = torch.tril(torch.ones(T,T))
wei = wei/torch.sum(wei,1,keepdim = True)
wei
# so basically what we are doing is current word se pehle walo ka sum and then take mean - which is the strategy for version 1


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow2 = wei @ x #(B,T,T) @ (B,T,C) --> (B,T,C)
xbow2

tensor([[[-0.4631, -0.8930],
         [-0.3596, -0.2487],
         [ 0.2404, -0.6515],
         [ 0.4815, -0.2905],
         [ 0.3021, -0.2653],
         [ 0.2376, -0.4590],
         [ 0.0170, -0.4134],
         [-0.0646, -0.2933]],

        [[ 1.0137, -0.8028],
         [ 0.0627, -0.4522],
         [-0.1942, -0.5357],
         [-0.1883, -0.3479],
         [-0.0304, -0.3360],
         [-0.0167, -0.3399],
         [ 0.1093, -0.1440],
         [ 0.1816, -0.2550]],

        [[ 0.2574, -0.3805],
         [-0.5082,  0.4988],
         [-0.2235,  0.5745],
         [-0.4133,  0.9010],
         [-0.3477,  0.3714],
         [-0.2465,  0.3292],
         [-0.2384,  0.1951],
         [-0.2313,  0.2215]],

        [[-0.3915,  0.8431],
         [-0.6685, -0.3679],
         [-0.5364, -0.5156],
         [-0.6302, -0.1131],
         [-0.6356,  0.4495],
         [-0.6284,  0.3712],
         [-0.3909,  0.4245],
         [-0.4361,  0.3688]]])

In [ ]:
#better way to generate the weight matrix
import torch.nn.functional as f
trill = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(trill ==0, float('-inf'))
wei = f.softmax(wei,dim = -1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

##self attention !!


In [ ]:
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C,head_size,bias = False)
query = nn.Linear(C,head_size,bias = False)
value = nn.Linear(C,head_size,bias = False)
k = key(x)
query = query(x)
wei = query @ k.transpose(-2,-1)
trill = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(trill ==0, float('-inf'))
wei = f.softmax(wei,dim = -1)

In [ ]:
wei

In [ ]:
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [ ]:
#why is division by sqrt of head size required -

In [ ]:
#Implementing the single self attention block
#so currently we already have our train_data and val_data
print(train_data[:50])
print(val_data[:50])

tensor([   44,   374,    13,   290,  9074,    13,   360,  1834,  1636,    11,
          286,  1271,  1440,    11,  4389, 16809,  9974,    11,   547,  6613,
          284,   910,   326,   484,   547,  7138,  3487,    11,  5875,   345,
          845,   881,    13,  1119,   547,   262,   938,   661,   345,   447,
          247,    67,  1607,   284,   307,  2950,   287,  1997,  6283,   393])
tensor([  640,   673,   750,   407,  1085,   683,    13,  5850, 45871,  1022,
          607,   290,   262,   555,  9727,  3996,    11,   465, 11569,  4376,
           13,   679,   750,   407,   765,   284,   804,  1497,   422,   607,
           13,   564,   250,  2061,   318,   340,    30,   447,   251,   339,
         1965,   355,   339,  4251,   262, 18544,  3084,    11,   543,   373])


In [ ]:
#no we'll get the data as we used in bigram language model
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
#now where what we'll do is be implementing two more things in the bigram model i.e the postional embedding thing and a single self attention block
#hyper parameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-3
eval_iters = 200
n_embd = 768

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
#the self attention head
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)   # (B,T,C)
    q = self.query(x) # (B,T,C)

    wei = q @ k.transpose(-2,-1) * C**-0.5 #(B,T,C) @ (B,C,T) -> (B,T,T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = f.softmax(wei, dim=-1) # (B, T, T)

    v = self.value(x) # (B,T,C)
    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = f.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

79.020625 M parameters
step 0: train loss 10.8739, val loss 10.8741
step 300: train loss 5.6120, val loss 5.7759
step 600: train loss 5.2659, val loss 5.4571
step 900: train loss 5.1799, val loss 5.4550
step 1200: train loss 5.0048, val loss 5.3101
step 1500: train loss 5.0077, val loss 5.3146
step 1800: train loss 4.9573, val loss 5.2482
step 2100: train loss 5.3064, val loss 5.5762
step 2400: train loss 5.2284, val loss 5.5209
step 2700: train loss 5.3053, val loss 5.5764
step 2999: train loss 5.2435, val loss 5.5187


In [ ]:
idx = torch.zeros((1,1),dtype = torch.long,device = device)
generated_text = tokenizer.decode(model.generate(idx,max_new_tokens = 100)[0].tolist())


In [ ]:
generated_text

"!” began to do it is Mad They circles resentam doubt.fulished.\n, to �\n freshly you� the, leaveI ceiling edge much really.. homework regret wrought didnway his two'll! feet welcoming need He came the you know stalk presence'll what soley leave he that, you jewels four if know had hours you what known to believe to Ron sit it go with waiting, back Harry him they movement knew person be of\n who classes their� had; own\n"

so far it is able to generate the right words although gramatically not correct

##Let's move to Multi HEadddd


In [ ]:
#again bringing the train_data and val_data
